In [30]:
################
# Data pre-processing
# 1. KW composite
# 2. Calculate EAPE/EKE growth rate
# 3. Calculate regional/seasonal mean of SST, U200
# 2024.9.16
# Mu-Ting Chien
#########################

In [31]:
import numpy as np
from netCDF4 import Dataset
import sys
sys.path.append('/home/disk/eos4/muting/function/python/')
import KW_diagnostics as KW
import mjo_mean_state_diagnostics as MJO

In [35]:
DIR = '/home/disk/eos4/muting/'
dir_out = DIR+'KW/'
#################################
# Copied directory:
dir_in = dir_out+'output_data/RA_EAPEG_EKEG/paper3_final_20240910/'
dir_in_org = dir_in+'basic_data/'
###########################
# Original directory:
#dir_in0 = DIR+'mesoscale_CCEW/'
#dir_out2_org = dir_out+'output_data/RA_EAPEG_EKEG/3hr/final/'
####################################
trange = '1983_2013' #file name for KW phase
pi = np.pi
ifr = 0
freqname = list([''])
ymin  = 1998
nyr_leap = 5 # 5 leap years between 1980 and ymin
season  = list(['DJF','MAM','JJA','SON','ANN'])
region  = list(['IO','WP-CP','EP','SA-ATL','AFR'])
nsea = np.size(season)
nreg = np.size(region)
lon_min = np.array([55, 130, 220, 275, 345])
lon_max = np.array([130, 220, 275, 345, 50])
lon_ref = np.array([55,130,220,275,345]) # marking the boundary of each region
nlon1 = int( (360-lon_min[-1])/2.5) # left boundary of AFRICA
nlon2 = int(lon_max[-1]/2.5+1 ) # right boundary of AFRICA
nlon_afr = int( nlon1 + nlon2 )
nlon1_sst = int( (360.5-lon_min[-1])) # left boundary of AFRICA
nlon2_sst = int(lon_max[-1]+1+0.5 ) # right boundary of AFRICA
nlon_afr_sst = int( nlon1_sst + nlon2_sst )

In [33]:
#################################
# Load data for 1. KW composite
####################################
# Load KW phase
#file   = dir_in0+'output_data/phase_CCKW_filt_Tb_'+trange+freqname[ifr]+'.nc'
file   = dir_in_org+'phase_CCKW_filt_Tb_'+trange+freqname[ifr]+'.nc'
data   = Dataset(file,'r',format='NETCDF4')
time_kw  = data.variables['time'][:]
tmin_day = (ymin-1980)*365+nyr_leap
itmin    = np.argwhere(time_kw==tmin_day).squeeze()
lat      = data.variables['lat'][:]
ilat_eq  = np.argwhere(lat==0).squeeze()
phase_kw = data.variables['phase_kw'][itmin:,ilat_eq,:] #(time, lon) #(lat = -15~15, lon=0~360) Only use equator to represent the phase
time_kw = time_kw[itmin:]

phase_kw = phase_kw[8:-8,:]
time_kw = time_kw[8:-8]

In [36]:
# Load meridional projected 2D variables (LH & precip)
#file = dir_out2_org+'merdional_proj_LHpr_remove_lowpass.nc'
file = dir_in_org+'merdional_proj_LHpr_remove_lowpass.nc'
data = Dataset(file, 'r', format='NETCDF4')
LH_proj = data.variables['LH_proj'][:]
pr_proj = data.variables['pr_proj'][:]

# Load phase bin
#data = np.load(dir_out2_org+'KW_composite_2D_LHpr.npz')
data = np.load(dir_in+'KW_composite_2D_LHpr.npz')
phase = data['phase']
nphase = np.size(phase)

In [38]:
# Load KW filtered Tb
#file   = dir_in0+'output_data/Tb_15NS_CCKW_filt_1983_2013_3hr.nc'
file   = dir_in+'Tb_15NS_CCKW_filt_1983_2013_3hr.nc'
data   = Dataset(file,'r',format='NETCDF4')
time_kw  = data.variables['time'][:]
lon      = data.variables['lon'][:]
tmin_day = (ymin-1980)*365+nyr_leap
itmin    = np.argwhere(time_kw==tmin_day).squeeze()
lat      = data.variables['lat'][:]
Tb_kw    = data.variables['Tb_kw'][itmin:,:,:]
time_kw  = time_kw[itmin:]
time_kw  = time_kw[8:-8] # This is to make sure having the same time dimension as the composited variable (u, F, q...)
Tb_kw    = Tb_kw[8:-8,:,:]

# Tb std for each longitude and latitude (only within 10S to 10N)
Tb_std = np.std(Tb_kw,0) #(lon, lat)

# Meridional projection of Tb
Tb_proj = KW.KW_meridional_projection(Tb_kw, lat, tropics_or_midlat=0)

In [39]:
# Load meridional projected 3D variables (QTFuqw)
#file = dir_out2_org+'merdional_proj_3d_remove_lowpass.nc'  
file = dir_in_org+'merdional_proj_3d_remove_lowpass.nc'  
data = Dataset(file, 'r', format='NETCDF4')
T_proj = data.variables['T_proj'][:,1:-1,:] #(time, plev, lon), plev 1:-1 is to make sure this is consistent with EOF
u_proj = data.variables['u_proj'][:,1:-1,:]
q_proj = data.variables['q_proj'][:,1:-1,:]*1000
F_proj = data.variables['F_proj'][:,1:-1,:]
Q_proj = data.variables['Q_proj'][:,1:-1,:]
w_proj = data.variables['w_proj'][:,1:-1,:]
z_proj = data.variables['gph_proj'][:,1:-1,:]
t_proj = data.variables['time'][:]
plev_proj = data.variables['plev'][1:-1]
nlev = np.size(plev_proj)

In [40]:
# Load time data
#file = dir_out2_org+'merdional_proj_LHpr_remove_lowpass.nc'
file = dir_in_org+'merdional_proj_LHpr_remove_lowpass.nc'
data = Dataset(file, 'r', format='NETCDF4')
time = data.variables['time'][:]
time_ymd = np.floor(time/100)
lon = data.variables['lon'][:]

In [41]:
####################################
# 3. Load data for EOF decomposition
##################################
# Load EOF
#file_in  = dir_out+'EOF_QWU_updated.npz' 
file_in  = dir_in_org+'EOF_QWU_updated.npz' 
data     = np.load(file_in)
EOF1_Q   = data['EOF1_Q']
EOF2_Q   = data['EOF2_Q']
EOF1_U   = data['EOF1_U']
EOF2_U   = data['EOF2_U']
plev_eof = data['plev']

In [ ]:
########################################
# 4. Load sst data (monthly data since 187001) and calculate mean SST
##############################################
# Load SST 
ymin = 1998
ymax = 2013
nyr  = ymax-ymin+1
tmin = (ymin-1870)*12
tmax = tmin+nyr*12

#file   = DIR+'data/HadISST.nc'
file   = dir_in_org+'HadISST.nc'
data   = Dataset(file,'r',format='NETCDF4')
lat_sst_old = data.variables['latitude'][:]
lon_sst_old = data.variables['longitude'][:]
ilatmin = np.argwhere(lat_sst_old==30.5).squeeze()
ilatmax = np.argwhere(lat_sst_old==-30.5).squeeze()
sst_old = data.variables['sst'][tmin:tmax,ilatmin:ilatmax+1,:]

# rearrange longitude and latitude
lat_sst = lat_sst_old[ilatmin:ilatmax+1][::-1]
sst_old = sst_old[:,::-1,:]
nlon_sst = np.size(lon_sst_old)
nlat_sst = np.size(lat_sst)
nt_sst   = np.size(sst_old,0)
#
nlon_sst_mid       = int(nlon_sst/2)
sst                = np.empty([nt_sst, nlat_sst, nlon_sst])
sst[:,:,nlon_sst_mid:] = sst_old[:,:,:nlon_sst_mid]
sst[:,:,:nlon_sst_mid] = sst_old[:,:,nlon_sst_mid:]
#
lon_sst            = np.empty([nlon_sst])
lon_sst[nlon_sst_mid:] = lon_sst_old[:nlon_sst_mid]
lon_sst[:nlon_sst_mid] = lon_sst_old[nlon_sst_mid:]

lon_sst = np.where(lon_sst<0, lon_sst+360, lon_sst)

# Calcualte seasonal mean
month = np.arange(1,13,1)
mon_sst = matlib.repmat(month,nyr,1)
#
yr_sst = np.arange(ymin, ymax+1)
yr_sst = matlib.repmat(yr_sst,12,1).T
#
time_sst = yr_sst*10000+mon_sst*100+1
time_sst = np.reshape(time_sst,12*nyr)

# Calculate time mean for each season
sst_sea = np.empty([nlat_sst, nlon_sst, nsea])
for isea in range(0,nsea):
    MonID               = season[isea]
    #
    tmp, t_sea          = MJO.Seasonal_Selection(sst[:,:,:], time_sst, MonID, 0)
    tmp                 = np.where(tmp<-10**29, np.nan, tmp) # make sure filled value become nan
    sst_sea[:,:,isea]   = np.nanmean(tmp,0)

In [ ]:
###################################
# 1. KW composite Tb and precip
##################################
calc_kw_composite = 1
if calc_kw_composite == 1:
    
    # Select basins and seasons

    # Initialize
    kw_number   = np.empty([nreg, nsea])
    pr_KW_small = np.empty([nphase, nreg, nsea])
    Tb_KW_small = np.empty([nphase, nreg, nsea])
    LH_KW_small = np.empty([nphase, nreg, nsea])
    T_KW_small  = np.empty([nphase, nlev, nreg, nsea])
    Q_KW_small  = np.empty([nphase, nlev, nreg, nsea])
    F_KW_small  = np.empty([nphase, nlev, nreg, nsea])
    U_KW_small  = np.empty([nphase, nlev, nreg, nsea])
    q_KW_small  = np.empty([nphase, nlev, nreg, nsea])

    for ireg in range(0,nreg):
        ilon_min = np.argwhere(lon==lon_min[ireg]).squeeze()
        ilon_max = np.argwhere(lon==lon_max[ireg]).squeeze()
        
        if ireg != nreg-1:
            # 2D variables:
            pr_proj_small = pr_proj[:,ilon_min:ilon_max+1]
            Tb_proj_small = Tb_proj[:,ilon_min:ilon_max+1]
            LH_proj_small = LH_proj[:,ilon_min:ilon_max+1]
            # 3D variables:
            T_proj_small  = T_proj[:,:,ilon_min:ilon_max+1]
            Q_proj_small  = Q_proj[:,:,ilon_min:ilon_max+1]
            u_proj_small  = u_proj[:,:,ilon_min:ilon_max+1]
            F_proj_small  = F_proj[:,:,ilon_min:ilon_max+1]
            q_proj_small  = q_proj[:,:,ilon_min:ilon_max+1]
            # KW phase:
            phase_kw_small = phase_kw[:,ilon_min:ilon_max+1]
        else:
            nt = np.size(pr_proj,0)
            pr_proj_small = np.empty([nt,nlon_afr])
            Tb_proj_small = np.empty([nt,nlon_afr])
            LH_proj_small = np.empty([nt,nlon_afr])
            T_proj_small  = np.empty([nt,nlev,nlon_afr])
            Q_proj_small  = np.empty([nt,nlev,nlon_afr])
            u_proj_small  = np.empty([nt,nlev,nlon_afr])
            F_proj_small  = np.empty([nt,nlev,nlon_afr])
            q_proj_small  = np.empty([nt,nlev,nlon_afr])
            phase_kw_small= np.empty([nt,nlon_afr])
            lon_small     = np.empty([nlon_afr])
            #################
            # lon = 345-360
            # 2D variables:
            #print(ilon_min, ilon_max)
            lon_small[:nlon1] = lon[ilon_min:]
            pr_proj_small[:,:nlon1] = pr_proj[:,ilon_min:]
            Tb_proj_small[:,:nlon1] = Tb_proj[:,ilon_min:]
            LH_proj_small[:,:nlon1] = LH_proj[:,ilon_min:]
            # 3D variables:
            T_proj_small[:,:,:nlon1]  = T_proj[:,:,ilon_min:]
            Q_proj_small[:,:,:nlon1]  = Q_proj[:,:,ilon_min:]
            u_proj_small[:,:,:nlon1]  = u_proj[:,:,ilon_min:]
            F_proj_small[:,:,:nlon1]  = F_proj[:,:,ilon_min:]
            q_proj_small[:,:,:nlon1]  = q_proj[:,:,ilon_min:]
            # KW phase:
            phase_kw_small[:,:nlon1] = phase_kw[:,ilon_min:]     
            ################################
            # lon = 0-50
            # 2D variables:
            lon_small[nlon1:] = lon[:ilon_max+1]
            #print(lon_small)
            pr_proj_small[:,nlon1:] = pr_proj[:,:ilon_max+1]
            Tb_proj_small[:,nlon1:] = Tb_proj[:,:ilon_max+1]
            LH_proj_small[:,nlon1:] = LH_proj[:,:ilon_max+1]
            # 3D variables:
            T_proj_small[:,:,nlon1:]  = T_proj[:,:,:ilon_max+1]
            Q_proj_small[:,:,nlon1:]  = Q_proj[:,:,:ilon_max+1]
            u_proj_small[:,:,nlon1:]  = u_proj[:,:,:ilon_max+1]
            F_proj_small[:,:,nlon1:]  = F_proj[:,:,:ilon_max+1]
            q_proj_small[:,:,nlon1:]  = q_proj[:,:,:ilon_max+1]
            # KW phase:
            phase_kw_small[:,nlon1:] = phase_kw[:,:ilon_max+1] 
        
        for isea in range(0,5):
            MonID = season[isea]

            # KW phase:
            phase_kw_sea, t_sea = MJO.Seasonal_Selection(phase_kw_small, time_ymd, MonID, 0)

            # 2D vairables:
            V_sea, t_sea                     = MJO.Seasonal_Selection(pr_proj_small, time_ymd, MonID, 0)
            pr_KW_small[:,ireg,isea], phase  = KW.composite_by_kw_phase(V_sea, phase_kw_sea, t_sea, std=0)
            #
            V_sea, t_sea                     = MJO.Seasonal_Selection(Tb_proj_small, time_ymd, MonID, 0)
            Tb_KW_small[:,ireg,isea], phase  = KW.composite_by_kw_phase(V_sea, phase_kw_sea, t_sea, std=0)
            #
            V_sea, t_sea                     = MJO.Seasonal_Selection(LH_proj_small, time_ymd, MonID, 0)
            LH_KW_small[:,ireg,isea], phase  = KW.composite_by_kw_phase(V_sea, phase_kw_sea, t_sea, std=0)

            # 3D variables:
            V_sea, t_sea                     = MJO.Seasonal_Selection(T_proj_small, time_ymd, MonID, 0)
            T_KW_small[:,:,ireg,isea], phase = KW.composite_by_kw_phase(V_sea, phase_kw_sea, t_sea, std=0)
            #
            V_sea, t_sea                     = MJO.Seasonal_Selection(Q_proj_small, time_ymd, MonID, 0)
            Q_KW_small[:,:,ireg,isea], phase = KW.composite_by_kw_phase(V_sea, phase_kw_sea, t_sea, std=0)
            #
            V_sea, t_sea                     = MJO.Seasonal_Selection(F_proj_small, time_ymd, MonID, 0)
            F_KW_small[:,:,ireg,isea], phase = KW.composite_by_kw_phase(V_sea, phase_kw_sea, t_sea, std=0)
            #
            V_sea, t_sea                     = MJO.Seasonal_Selection(u_proj_small, time_ymd, MonID, 0)
            U_KW_small[:,:,ireg,isea], phase = KW.composite_by_kw_phase(V_sea, phase_kw_sea, t_sea, std=0)
            #
            V_sea, t_sea                     = MJO.Seasonal_Selection(q_proj_small, time_ymd, MonID, 0)
            q_KW_small[:,:,ireg,isea], phase = KW.composite_by_kw_phase(V_sea, phase_kw_sea, t_sea, std=0)

            # Count KW case number:
            active_kw = np.where(phase_kw_sea==-pi/2, 1, np.nan)
            kw_number[ireg,isea] = np.nansum(active_kw)
    
    # Save regional KW composite
    np.savez(dir_in+'kwcomposite_2D_region_season.npz',region=region, season=season,\
         pr_KW_small=pr_KW_small, LH_KW_small=LH_KW_small, Tb_KW_small=Tb_KW_small, kw_number=kw_number)

    # Save 3d data: note that the unit of q is g/kg
    np.savez(dir_in+'kwcomposite_3D_region_season.npz',region=region, season=season, plev=plev_proj,\
         T_KW_small=T_KW_small, Q_KW_small=Q_KW_small, F_KW_small=F_KW_small, U_KW_small=U_KW_small, q_KW_small=q_KW_small)
else:
    data        = np.load(dir_in+'kwcomposite_2D_region_season.npz')
    region      = data['region']
    season      = data['season']
    pr_KW_small = data['pr_KW_small']
    LH_KW_small = data['LH_KW_small']
    Tb_KW_small = data['Tb_KW_small']
    kw_number   = data['kw_number']
    #
    data       = np.load(dir_in+'kwcomposite_3D_region_season.npz')
    plev       = data['plev']
    T_KW_small = data['T_KW_small']
    Q_KW_small = data['Q_KW_small']
    F_KW_small = data['F_KW_small']
    U_KW_small = data['U_KW_small']
    q_KW_small = data['q_KW_small']

In [19]:
#########################################
# 2. Calculate vertical mode decomposition
#########################################
calc_vmd = 1
if calc_vmd == 1:
    
    for ireg in range(0,nreg):  
        for isea in range(0,nsea):
            if ireg == 0 and isea == 0:
                Q1_KW_small = np.empty([nphase, nlev, nreg, nsea])
                Q2_KW_small = np.empty([nphase, nlev, nreg, nsea])
                T1_KW_small = np.empty([nphase, nlev, nreg, nsea])
                T2_KW_small = np.empty([nphase, nlev, nreg, nsea])
                F1_KW_small = np.empty([nphase, nlev, nreg, nsea])
                F2_KW_small = np.empty([nphase, nlev, nreg, nsea])
                U1_KW_small = np.empty([nphase, nlev, nreg, nsea])
                U2_KW_small = np.empty([nphase, nlev, nreg, nsea])
            Q1_KW_small[:,:,ireg,isea], Q2_KW_small[:,:,ireg,isea] = KW.vertical_mode_decomposition(Q_KW_small[:,:,ireg,isea], EOF1_Q, EOF2_Q)
            T1_KW_small[:,:,ireg,isea], T2_KW_small[:,:,ireg,isea] = KW.vertical_mode_decomposition(T_KW_small[:,:,ireg,isea], EOF1_Q, EOF2_Q)
            F1_KW_small[:,:,ireg,isea], F2_KW_small[:,:,ireg,isea] = KW.vertical_mode_decomposition(F_KW_small[:,:,ireg,isea], EOF1_U, EOF2_U)
            U1_KW_small[:,:,ireg,isea], U2_KW_small[:,:,ireg,isea] = KW.vertical_mode_decomposition(U_KW_small[:,:,ireg,isea], EOF1_U, EOF2_U)
    
    # Save data
    np.savez(dir_in+'kwcomposite_3D_region_season_QTFU_vmd_updated.npz',region=region, season=season, plev=plev,\
         Q1_KW_small=Q1_KW_small, T1_KW_small=T1_KW_small, F1_KW_small=F1_KW_small, U1_KW_small=U1_KW_small,\
         Q2_KW_small=Q2_KW_small, T2_KW_small=T2_KW_small, F2_KW_small=F2_KW_small, U2_KW_small=U2_KW_small)

In [20]:
######################################
# 3. Calculate EAPE/EKE growth rate
######################################
calculate_gr = 1
s2d = 86400
if calculate_gr == 1:
    for ireg in range(0,nreg):
        for isea in range(0,nsea):
            PC1_Q_small = np.inner(Q_KW_small[:,:,ireg,isea], EOF1_Q) #(nphase)
            PC2_Q_small = np.inner(Q_KW_small[:,:,ireg,isea], EOF2_Q)
            PC1_T_small = np.inner(T_KW_small[:,:,ireg,isea], EOF1_Q) 
            PC2_T_small = np.inner(T_KW_small[:,:,ireg,isea], EOF2_Q)

            PC1_F_small = np.inner(F_KW_small[:,:,ireg,isea], EOF1_U) 
            PC2_F_small = np.inner(F_KW_small[:,:,ireg,isea], EOF2_U)
            PC1_U_small = np.inner(U_KW_small[:,:,ireg,isea], EOF1_U) 
            PC2_U_small = np.inner(U_KW_small[:,:,ireg,isea], EOF2_U)

            if ireg == 0 and isea == 0:
                EAPEGR1 = np.empty([nreg,nsea])
                EAPEGR2 = np.empty([nreg,nsea])
                EKEGR1  = np.empty([nreg,nsea])
                EKEGR2  = np.empty([nreg,nsea])

            EAPEGR1[ireg,isea] = 2*np.mean(PC1_Q_small*PC1_T_small)/np.mean(PC1_T_small**2)
            EAPEGR2[ireg,isea] = 2*np.mean(PC2_Q_small*PC2_T_small)/np.mean(PC2_T_small**2)
            EKEGR1[ireg,isea]  = 2*np.mean(PC1_F_small*PC1_U_small)/np.mean(PC1_U_small**2)*s2d
            EKEGR2[ireg,isea]  = 2*np.mean(PC2_F_small*PC2_U_small)/np.mean(PC2_U_small**2)*s2d
            
    # Calculate relative contribution of internal vs. external, just looking at the second mode
    ratio_gr2 = EKEGR2/EAPEGR2
    
    # Save growth rate
    np.savez(dir_in+'EAPE_EKE_growth_rate_region_season_updated.npz',region=region, season=season,\
         EAPEGR1=EAPEGR1, EAPEGR2=EAPEGR2, EKEGR1=EKEGR1, EKEGR2=EKEGR2, ratio_gr2=ratio_gr2)

In [ ]:
##########################
# 4-1. Calculate mean U
#########################
# Load mean state U
file   = dir_in+'regrid_u.nc'
data   = Dataset(file,'r',format='NETCDF4')
lat    = data.variables['lat'][:]
lon_old= data.variables['lon'][:]
nlon   = np.size(lon)
plev   = data.variables['level'][1:-1]
nlev   = np.size(plev)

########################
# Select 10-20 S/N (Testing only)
ilatmax2 = np.argwhere(lat==20).squeeze().squeeze()
ilatmin2 = np.argwhere(lat==-20).squeeze().squeeze()
u_10_20N_org = data.variables['u'][8:-8,1:-1,ilatmax:ilatmax2+1,:]
u_10_20S_org = data.variables['u'][8:-8,1:-1,ilatmin2:ilatmin+1,:]
lat_10_20N = lat[ilatmax:ilatmax2+1]
lat_10_20S = lat[ilatmin2:ilatmin+1]
nlat_10_20 = np.size(lat_10_20N)
###############################

# Select 10S-10N only
ilatmin = np.argwhere(lat==-10).squeeze()
ilatmax = np.argwhere(lat==10).squeeze()
lat = lat[ilatmin:ilatmax+1]
nlat   = np.size(lat)
u_old  = data.variables['u'][8:-8,1:-1,ilatmin:ilatmax+1,:] #time is reduced to match the Qproj, Tproj...
nt     = np.size(u_old,0)

# Rearrange u longitude
nlon_mid           = int(nlon/2)
u                  = np.empty([nt, nlev, nlat, nlon])
u[:,:,:,nlon_mid:] = u_old[:,:,:,:nlon_mid]
u[:,:,:,:nlon_mid] = u_old[:,:,:,nlon_mid:]

u_10_20N                  = np.empty([nt, nlev, nlat_10_20, nlon])
u_10_20N[:,:,:,nlon_mid:] = u_10_20N_org[:,:,:,:nlon_mid]
u_10_20N[:,:,:,:nlon_mid] = u_10_20N_org[:,:,:,nlon_mid:]

u_10_20S                  = np.empty([nt, nlev, nlat_10_20, nlon])
u_10_20S[:,:,:,nlon_mid:] = u_10_20S_org[:,:,:,:nlon_mid]
u_10_20S[:,:,:,:nlon_mid] = u_10_20S_org[:,:,:,nlon_mid:]

# Calculate time mean and latitudinal mean (10S-10N mean)
for isea in range(0,nsea):
    MonID              = season[isea]
    u_sea, t_sea       = MJO.Seasonal_Selection(u, time_ymd, MonID, 0)
    u_sea_10_20N, tsea = MJO.Seasonal_Selection(u_10_20N, time_ymd, MonID, 0)
    u_sea_10_20S, tsea = MJO.Seasonal_Selection(u_10_20S, time_ymd, MonID, 0)
    u_tm               = np.mean(u_sea,0) #time mean
    u_tm_10_20N        = np.mean(u_sea_10_20N,0) #time mean
    u_tm_10_20S        = np.mean(u_sea_10_20S,0) #time mean
    if isea == 0:
        u_tym = np.empty([nlev,nlon,nsea])
        u_tym_10_20 = np.empty([nlev,nlon,nsea]) # only NH in DJF, SH in JJA, NH and SH in other seasons
        u_tym_10_20_SN = np.empty([nlev,nlon,nsea]) # both SH and NH
    u_tym[:,:,isea] = MJO.mer_ave(u_tm, lat, 1) #time mean and latitudinal mean

    if isea == 0:
        u_tym_10_20[:,:,isea] = MJO.mer_ave(u_tm_10_20N, lat_10_20N, 1) # DJF, use NH only
    elif isea == 2:
        u_tym_10_20[:,:,isea] = MJO.mer_ave(u_tm_10_20S, lat_10_20S, 1) # JJA, use SH only
    else:
        u_tym_10_20[:,:,isea] = 1/2*( MJO.mer_ave(u_tm_10_20S, lat_10_20S, 1) + MJO.mer_ave(u_tm_10_20N, lat_10_20N, 1) )
        
    u_tym_10_20_SN[:,:,isea] = 1/2*( MJO.mer_ave(u_tm_10_20S, lat_10_20S, 1) + MJO.mer_ave(u_tm_10_20N, lat_10_20N, 1) )

In [ ]:
####################################
# 4-2. Find magnitude of T_KW, U_KW, Q_KW, F_KW
##################################################
F_amp_KW = (np.max(F_KW_small,0)-np.min(F_KW_small,0))/2
U_amp_KW = (np.max(U_KW_small,0)-np.min(U_KW_small,0))/2
Q_amp_KW = (np.max(Q_KW_small,0)-np.min(Q_KW_small,0))/2
T_amp_KW = (np.max(T_KW_small,0)-np.min(T_KW_small,0))/2

np.savez(dir_in+'MeanU_KW_amplitude_F_U_Q_T_plev_lon.npz', \
         F_amp_KW=F_amp_KW, U_amp_KW=U_amp_KW, Q_amp_KW=Q_amp_KW, T_amp_KW=T_amp_KW, mean_U = u_tym, \
         season=season, lon=lon, plev=plev,\
        mean_U_10_20 = u_tym_10_20, mean_U_10_20_SN = u_tym_10_20_SN)

##############################
# 4-3. Calculate regional mean of KW amplitude
region  = list(['IO','WP-CP','EP-ATL','AFR','GLO'])
nreg = np.size(region)
lon_min = np.array([55, 130, 250, 345, 0])
dx      = 2.5
lon_max = np.array([130-dx, 250-dx, 345-dx, 55-dx, 360-dx])
nlon_afr = (360-dx-lon_min[3]+lon_max[3])/dx+2
print(nlon_afr)
for ireg in range(0,nreg):
    
    if ireg !=3:
        ilon_min = np.argwhere(lon==lon_min[ireg]).squeeze()
        ilon_max = np.argwhere(lon==lon_max[ireg]).squeeze()
    else:
        ilon_min = np.argwhere(lon==lon_max[ireg]).squeeze()
        ilon_max = np.argwhere(lon==lon_min[ireg]).squeeze()        
        
    if ireg == 0:
        Q_amp_KW_reg = np.empty([nlev,nreg,nsea])
        T_amp_KW_reg = np.empty([nlev,nreg,nsea])
        U_amp_KW_reg = np.empty([nlev,nreg,nsea])
        F_amp_KW_reg = np.empty([nlev,nreg,nsea])
        
    if ireg != 3:
        Q_amp_KW_reg[:,ireg,:] = np.mean(Q_amp_KW[:,ilon_min:ilon_max+1,:],1) 
        T_amp_KW_reg[:,ireg,:] = np.mean(T_amp_KW[:,ilon_min:ilon_max+1,:],1) 
        U_amp_KW_reg[:,ireg,:] = np.mean(U_amp_KW[:,ilon_min:ilon_max+1,:],1) 
        F_amp_KW_reg[:,ireg,:] = np.mean(F_amp_KW[:,ilon_min:ilon_max+1,:],1) 
    else:
        Q_amp_KW_reg[:,ireg,:] = (np.sum(Q_amp_KW,1)-np.sum(Q_amp_KW[:,ilon_min:ilon_max+1,:],1))/nlon_afr
        T_amp_KW_reg[:,ireg,:] = (np.sum(T_amp_KW,1)-np.sum(T_amp_KW[:,ilon_min:ilon_max+1,:],1))/nlon_afr
        U_amp_KW_reg[:,ireg,:] = (np.sum(U_amp_KW,1)-np.sum(U_amp_KW[:,ilon_min:ilon_max+1,:],1))/nlon_afr
        F_amp_KW_reg[:,ireg,:] = (np.sum(F_amp_KW,1)-np.sum(F_amp_KW[:,ilon_min:ilon_max+1,:],1))/nlon_afr
        
np.savez(dir_in+'KW_amplitude_F_U_Q_T_plev_regional_mean.npz', \
         F_amp_KW_reg=F_amp_KW_reg, U_amp_KW_reg=U_amp_KW_reg, Q_amp_KW_reg=Q_amp_KW_reg, T_amp_KW_reg=T_amp_KW_reg,\
         season=season, region=region, plev=plev)

In [ ]:
##############################
# 5. Calculate SST, U200 regional mean
#####################################

# Calculate 10S-10N, regional mean
for ireg in range(0,nreg):
    if ireg == 0:
        sst_region_mean  = np.empty([nreg,nsea])
        u200_region_mean = np.empty([nreg,nsea])
    
    ilat_min = np.argwhere(lat==-10).squeeze()
    ilat_max = np.argwhere(lat==10).squeeze()
    lat_tropics = lat[ilat_min:ilat_max+1]

    ilat_min_sst = np.argwhere(lat_sst==-9.5).squeeze()
    ilat_max_sst = np.argwhere(lat_sst==9.5).squeeze()
    lat_tropics_sst = lat_sst[ilat_min_sst:ilat_max_sst+1]
    
    ilon_min = np.argwhere(lon==lon_min[ireg]).squeeze()
    ilon_max = np.argwhere(lon==lon_max[ireg]).squeeze()
    ilon_min_sst = np.argwhere(lon_sst==lon_min[ireg]+0.5).squeeze()
    ilon_max_sst = np.argwhere(lon_sst==lon_max[ireg]+0.5).squeeze()
    
    nlat_small = np.size(lat_tropics)
    nlat_small_sst = np.size(lat_tropics_sst)
    
    if ireg != nreg-1:
        u200_small = u200_sea[ilat_min:ilat_max+1,ilon_min:ilon_max+1,:]
        sst_small  = sst_sea[ilat_min_sst:ilat_max_sst+1, ilon_min_sst:ilon_max_sst+1,:]
    else:
        u200_small = np.empty([nlat_small, nlon_afr, nsea])
        sst_small  = np.empty([nlat_small_sst, nlon_afr_sst, nsea])
        
        # lon = 345-360
        u200_small[:,:nlon1,:]    = u200_sea[ilat_min:ilat_max+1,ilon_min:,:]
        sst_small[:,:nlon1_sst,:] = sst_sea[ilat_min_sst:ilat_max_sst+1,ilon_min_sst:,:]
        
        # Lon = 0-50
        u200_small[:,nlon1:,:]    = u200_sea[ilat_min:ilat_max+1,:ilon_max+1,:]
        sst_small[:,nlon1_sst:,:] = sst_sea[ilat_min_sst:ilat_max_sst+1,:ilon_max_sst+1,:]
    

    #
    tmp = np.nanmean(u200_small,1)
    u200_region_mean[ireg,:] = MJO.mer_ave_2d(tmp, lat_tropics)
    #
    tmp = np.nanmean(sst_small,1)
    sst_region_mean[ireg,:] = MJO.mer_ave_2d(tmp, lat_tropics_sst)

# normalize
u200_norm = (u200_region_mean-np.mean(u200_region_mean))/np.std(u200_region_mean)
sst_norm = (sst_region_mean-np.mean(sst_region_mean))/np.std(sst_region_mean)

np.savez(dir_in+'SST_U200_regional_mean.npz', \
         u200_region_mean=u200_region_mean, sst_region_mean=sst_region_mean,\
        u200_norm=u200_norm, sst_norm=sst_norm, \
         u200_10_20_region_mean=u200_10_20_region_mean,\
        region=region)